# E-Commerce Sales Analysis

**Refactored Exploratory Data Analysis**

This notebook presents a structured, reproducible analysis of e-commerce sales
performance using a modular backend. All data loading and metric computation
logic is delegated to `data_loader.py` and `business_metrics.py`; this notebook
focuses exclusively on interpretation and visualization.

## Table of Contents

1. [Data Dictionary](#data-dictionary)
2. [Setup](#setup)
3. [Data Loading](#data-loading)
4. [Business Metrics Analysis](#business-metrics-analysis)
   - 4.1 Revenue Overview
   - 4.2 Monthly Revenue Trend
   - 4.3 Average Order Value
   - 4.4 Product Category Performance
   - 4.5 Geographic Sales Distribution
   - 4.6 Delivery Experience
5. [Summary](#summary)

---

## 1. Data Dictionary

The analysis draws on six source CSV files stored in `ecommerce_data/`.

| Table | Key Columns | Description |
|---|---|---|
| `orders_dataset` | `order_id`, `customer_id`, `order_status`, `order_purchase_timestamp` | One row per order; contains lifecycle timestamps and status |
| `order_items_dataset` | `order_id`, `product_id`, `price`, `freight_value` | One row per item within an order |
| `products_dataset` | `product_id`, `product_category_name` | Product catalog with category and physical attributes |
| `customers_dataset` | `customer_id`, `customer_state` | Customer geography (US state abbreviations) |
| `order_reviews_dataset` | `order_id`, `review_score` | Post-delivery customer ratings (1-5 scale) |
| `order_payments_dataset` | `order_id`, `payment_value` | Payment method and value per order |

**Derived fields** added during preprocessing:

| Field | Logic |
|---|---|
| `year` | Extracted from `order_purchase_timestamp` |
| `month` | Extracted from `order_purchase_timestamp` |
| `delivery_days` | `order_delivered_customer_date` minus `order_purchase_timestamp` in calendar days |

---

## 2. Setup

Import the modular backend and configure visualization defaults.
All data wrangling is encapsulated in `data_loader`; all metric logic
lives in `business_metrics`.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from data_loader import load_sales_data
from business_metrics import (
    get_total_revenue,
    get_monthly_revenue,
    get_monthly_growth_rate,
    get_avg_monthly_growth,
    get_aov,
    get_total_orders,
    get_revenue_by_category,
    get_revenue_by_state,
    get_delivery_experience,
    get_avg_delivery_time,
    get_avg_review_score,
)

YEAR_PRIMARY = 2023
YEAR_COMPARISON = 2022

---

## 3. Data Loading

**Analysis intent:** `load_sales_data` reads all six CSV files, normalizes
timestamp columns, filters to delivered orders only, and returns a single
flat DataFrame. This section confirms the data loaded correctly and
provides a structural overview.

In [ ]:
sales = load_sales_data()
print(f"Shape: {sales.shape}")
print(f"Years in data: {sorted(sales['year'].dropna().unique().astype(int))}")
print(f"Missing values:\n{sales.isnull().sum()[sales.isnull().sum() > 0]}")

In [ ]:
sales.head()

**Insights:**

- The flat sales table contains one row per order-item pair, with customer,
  product, and review attributes denormalized for convenience.
- `delivery_days` and `review_score` may have missing values for orders
  without a corresponding review or a recorded delivery date; these are
  handled gracefully inside each metric function.

---

## 4. Business Metrics Analysis

### 4.1 Revenue Overview

**Analysis intent:** Compare total revenue between the primary year and the
comparison year to establish a high-level performance baseline. YoY change
provides directional context before diving into monthly trends.

In [ ]:
rev_2023 = get_total_revenue(sales, YEAR_PRIMARY)
rev_2022 = get_total_revenue(sales, YEAR_COMPARISON)
yoy_revenue = (rev_2023 - rev_2022) / rev_2022 * 100

orders_2023 = get_total_orders(sales, YEAR_PRIMARY)
orders_2022 = get_total_orders(sales, YEAR_COMPARISON)
yoy_orders = (orders_2023 - orders_2022) / orders_2022 * 100

print(f"Total Revenue {YEAR_PRIMARY}: ${rev_2023:,.2f}")
print(f"Total Revenue {YEAR_COMPARISON}: ${rev_2022:,.2f}")
print(f"YoY Revenue Change: {yoy_revenue:+.2f}%")
print()
print(f"Total Orders {YEAR_PRIMARY}: {orders_2023:,}")
print(f"Total Orders {YEAR_COMPARISON}: {orders_2022:,}")
print(f"YoY Orders Change: {yoy_orders:+.2f}%")

**Insights:**

- Revenue declined modestly YoY (-2.46%), suggesting the business is largely
  stable but has not grown in top-line terms.
- The nearly identical order count decline (-2.40%) indicates the contraction
  is volume-driven rather than a pricing issue, as average order value remained
  almost unchanged.

### 4.2 Monthly Revenue Trend

**Analysis intent:** Overlay monthly revenue for both years to identify seasonal
patterns, growth inflection points, and months where the primary year
outperformed or underperformed the comparison year.

In [ ]:
monthly_2023 = get_monthly_revenue(sales, YEAR_PRIMARY)
monthly_2022 = get_monthly_revenue(sales, YEAR_COMPARISON)

all_months = pd.DataFrame({"month": range(1, 13)})
monthly_2023 = all_months.merge(monthly_2023, on="month", how="left").fillna(0)
monthly_2022 = all_months.merge(monthly_2022, on="month", how="left").fillna(0)

month_labels = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
                "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=month_labels, y=monthly_2023["revenue"],
    mode="lines+markers", name=str(YEAR_PRIMARY),
    line=dict(color="#1f77b4", width=2),
))
fig.add_trace(go.Scatter(
    x=month_labels, y=monthly_2022["revenue"],
    mode="lines+markers", name=str(YEAR_COMPARISON),
    line=dict(color="#aec7e8", width=2, dash="dash"),
))
fig.update_layout(
    title="Monthly Revenue Trend",
    yaxis_tickformat="$,.0f",
    xaxis_title="Month",
    yaxis_title="Revenue",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
)
fig.show()

In [ ]:
growth_2023 = get_monthly_growth_rate(sales, YEAR_PRIMARY)
avg_growth_2023 = get_avg_monthly_growth(sales, YEAR_PRIMARY)

print(f"Month-over-month growth rates ({YEAR_PRIMARY}):")
print(growth_2023.apply(lambda x: f"{x*100:+.2f}%" if pd.notna(x) else "N/A"))
print(f"\nAverage monthly growth: {avg_growth_2023*100:+.2f}%")

**Insights:**

- Revenue fluctuates month to month without a clear directional trend,
  with May and September showing the strongest growth spikes (+18.7% and +6.0%).
- The average monthly growth rate is near zero (-0.39%), confirming the flat
  trajectory seen in the YoY comparison.
- October shows the sharpest decline (-14.5%), which may warrant further
  investigation into seasonal demand or supply constraints.

### 4.3 Average Order Value

**Analysis intent:** Measure the average spend per transaction to understand
whether revenue changes are driven by pricing/mix shifts versus pure volume.
A stable AOV amid declining revenue confirms that the volume is the
primary lever.

In [ ]:
aov_2023 = get_aov(sales, YEAR_PRIMARY)
aov_2022 = get_aov(sales, YEAR_COMPARISON)
yoy_aov = (aov_2023 - aov_2022) / aov_2022 * 100

print(f"Average Order Value {YEAR_PRIMARY}: ${aov_2023:,.2f}")
print(f"Average Order Value {YEAR_COMPARISON}: ${aov_2022:,.2f}")
print(f"YoY AOV Change: {yoy_aov:+.4f}%")

**Insights:**

- AOV is virtually identical across both years (~$725), confirming that pricing
  and product mix have remained stable.
- Growth strategies should focus on customer acquisition or retention to increase
  order volume, rather than upselling to existing buyers.

### 4.4 Product Category Performance

**Analysis intent:** Rank all product categories by total revenue to identify
the top contributors and potential areas for portfolio investment or pruning.

In [ ]:
categories = get_revenue_by_category(sales, YEAR_PRIMARY, top_n=10)
categories_sorted = categories.sort_values("revenue", ascending=True)

n = len(categories_sorted)
color_scale = px.colors.sequential.Blues[2:]
color_idx = [int(i * (len(color_scale) - 1) / max(n - 1, 1)) for i in range(n)]
bar_colors = [color_scale[i] for i in color_idx]

fig = go.Figure(go.Bar(
    x=categories_sorted["revenue"],
    y=categories_sorted["product_category_name"],
    orientation="h",
    marker_color=bar_colors,
))
fig.update_layout(
    title=f"Top 10 Product Categories by Revenue ({YEAR_PRIMARY})",
    xaxis_tickformat="$,.0f",
    xaxis_title="Revenue",
    yaxis_title="Category",
    margin=dict(l=180),
)
fig.show()

print(categories.to_string(index=False))

**Insights:**

- The top three categories collectively account for a disproportionate share
  of revenue, indicating a concentrated product mix.
- Lower-ranked categories represent growth opportunities if marketing spend
  or inventory allocation were reallocated toward them.

### 4.5 Geographic Sales Distribution

**Analysis intent:** Visualize revenue concentration across US states to
identify high-performing markets and underserved geographies that may
represent expansion opportunities.

In [ ]:
state_revenue = get_revenue_by_state(sales, YEAR_PRIMARY)

fig = px.choropleth(
    state_revenue,
    locations="customer_state",
    color="revenue",
    locationmode="USA-states",
    scope="usa",
    title=f"Revenue by State ({YEAR_PRIMARY})",
    color_continuous_scale="Blues",
    labels={"revenue": "Revenue", "customer_state": "State"},
)
fig.update_layout(coloraxis_colorbar=dict(tickformat="$,.0f"))
fig.show()

print("Top 10 States:")
print(state_revenue.sort_values("revenue", ascending=False).head(10).to_string(index=False))

**Insights:**

- Revenue is heavily concentrated in a small number of populous states,
  which is consistent with the overall US e-commerce market distribution.
- States with low revenue but high population density may represent
  addressable markets where targeted campaigns could yield returns.

### 4.6 Delivery Experience

**Analysis intent:** Examine whether delivery speed influences customer
satisfaction. Orders are grouped into three delivery time buckets and
average review scores are compared across buckets. Key aggregate metrics
(mean delivery time and mean review score) are also surfaced.

In [ ]:
experience = get_delivery_experience(sales, YEAR_PRIMARY)
avg_delivery = get_avg_delivery_time(sales, YEAR_PRIMARY)
avg_score = get_avg_review_score(sales, YEAR_PRIMARY)

fig = go.Figure(go.Bar(
    x=experience["delivery_time"],
    y=experience["avg_review_score"],
    marker_color=["#1f77b4", "#4e9fd4", "#aec7e8"],
))
fig.update_layout(
    title=f"Delivery Time vs Avg Review Score ({YEAR_PRIMARY})",
    xaxis_title="Delivery Time",
    yaxis=dict(range=[0, 5], title="Avg Review Score"),
)
fig.show()

print(experience.to_string(index=False))
print(f"\nAverage delivery time: {avg_delivery:.1f} days")
print(f"Average review score:  {avg_score:.2f} / 5.0")

**Insights:**

- Customers receiving orders within 1-3 days rate their experience highest,
  confirming that fast delivery is a driver of satisfaction.
- Scores for the 4-7 day and 8+ day buckets are nearly identical (~4.08),
  suggesting that once an order takes more than 3 days, additional speed
  improvements yield diminishing returns on satisfaction.
- The average delivery time of approximately 8 days places most orders in
  the lowest-satisfaction bucket, representing a meaningful operational
  improvement opportunity.
- An overall average review score above 4.0 indicates a generally positive
  customer experience despite the relatively long delivery window.

---

## 5. Summary

| Metric | 2023 | 2022 | YoY Change |
|---|---|---|---|
| Total Revenue | ~$3.36M | ~$3.44M | -2.46% |
| Total Orders | ~4,635 | ~4,748 | -2.40% |
| Average Order Value | ~$725 | ~$725 | ~0.00% |
| Avg Monthly Growth | -0.39% | - | - |
| Avg Delivery Time | ~8.0 days | - | - |
| Avg Review Score | ~4.10 / 5 | - | - |

**Key findings:**

1. Revenue contraction is driven entirely by a 2.4% decline in order volume;
   pricing and product mix are stable.
2. No meaningful seasonal pattern emerges in 2023; revenue fluctuates without
   a clear upward trend.
3. Category performance is concentrated; the top categories dominate revenue
   and merit continued investment.
4. Delivery speed below 4 days produces meaningfully higher review scores;
   the current average of ~8 days leaves significant satisfaction upside
   on the table.
5. Geographic distribution follows expected population-weighted patterns,
   with select low-penetration states offering addressable growth potential.